### Chargement des modules nécessaires pour le projet
Commencez par exécuter la cellule ci-dessous. Elle va charger tout le nécessaire pour faire fonctionner votre code. 
Si vous le lancez en ligne, __soyez patients__ (attendez que l'étoile à gauche devienne un nombre entre crochets).


In [1]:
include("jupy.jl")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


main (generic function with 1 method)

### Donnéees à utiliser

On rappelle les structures qui sont remplies par les méthodes de lecture de données.

#### data structure for data used to compute the balance cost
```
struct BalanceData
    resource1::Int64        # first resource
    resource2::Int64        # second resource
    target::Int64           # target aimed for the recource consumption
    weight::Int64           # weight of the balance cost
end
```

#### data struture for an instance of Google challenge problem
```
struct DataGoogle
    nbResources::Int64               # number of resources in the instance
    transientStatus::Array{Int64}    # table indicating if each resource is transient (O/1)
    weightLoadCost::Array{Int64}     # table indicating the weight of load cost for each resource

    nbMachines::Int64                # number of machines
    neighborhoods::Array{Int64}      # table indicating the neighborhood of each machine
    nbNeighborhoods::Int64           # total number of neighbordhoods
    locations::Array{Int64}          # table indicating the location of each machine
    nbLocations::Int64               # total number of locations
    softCapacities::Array{Int64,2}   # softcapacities[m,r] indicates the soft cap of machine m for resource r
    hardCapacities::Array{Int64,2}   # hardcapacities[m,r] indicates the hard cap of machine m for resource r
    machineMoveCosts::Array{Int64,2} # machineMoveCosts[m1,m2] indicates the cost for moving from m1 to m2

    nbServices::Int64                # number of services
    spreadMin::Array{Int64}          # for each service, its spread min
    dependences::Array{Array{Int64}} # for each service, a list of dependences (service numbers)

    nbProcess::Int64                 # number of processes
    servicesProcess::Array{Int64}    # for each process  its service
    processReq::Array{Int64,2}       # processReq[p,r] indicates the consumption of resource r by p
    processMoveCost::Array{Int64}    # cost of moving each process p

    nbBalanceCostData::Int64               # number of balance constraints
    balanceCostDataList::Array{BalanceData} # one for each balance target
    processMoveWeight::Int64        # weight for the process move cost
    serviceMoveWeight::Int64        # weight for the service move cost
    machineMoveWeight::Int64        # weight for the machine move cost

    initialAssignment # for each process, id of the machine in the initial solution
end
```

#### slution 
```
struct SolutionGoogle
    assignment::Array{Int64}
    cost::Int64
end
```





### Cellule à modifier
Insérez ci-dessous votre modèle dans la méthode **solveGoogle**. 

N'oubliez pas de faire un run sur la cellule quand vous avez fini. 

In [2]:

using JuMP, Cbc  # JuMP  est la  bibliotheque  qui  sert à entrer  les  modèles
# Cbc est le  solver  utilise  pour  resoudre  les  modeles
function solveGoogle(data::DataGoogle, verbose::Bool)
   
    # Le programme a été codé en binôme par Clément Préaut et Ibrahim Lmourid
    
    #N.B. : Les @objective mis en commentaire ont servi à vérifier partiellement la correction des
    # contraintes et variables objectif par objectif (O1,O2,O3,O4,O5).
    
    #N.B : Les contraintes C5 et C6 ralentissent la résolution et ont donc été mises en commentaire.
    #Cependant, elle contiennent le résultat de nos recherches et constituent une proposition de réponse.
    
    m = Model(Cbc.Optimizer)    # on crée un modèle en précisant qu'on va utiliser Cbc
    
    
    @variable(m,x[i=1:data.nbMachines,j=1:data.nbProcess],Bin) # Processus j affecté à machine i (xij est un binaire)
    
    
    m0=data.initialAssignment  #machine initiale Sur laquelle est placé un processus donnné
    
    
    ###############################################
    #               Objectif 1
    ###############################################

    @variable(m,excess_SC[i=1:data.nbMachines,r=1:data.nbResources] >= 0,Int) 
    
    @variable(m,is_excess_SC[i=1:data.nbMachines,r=1:data.nbResources],Bin) 
    
    @constraint(m,init_excess_SC[i=1:data.nbMachines,r=1:data.nbResources],excess_SC[i,r] >= sum(data.processReq[p,r] * x[i,p] for p in 1:data.nbProcess) - data.softCapacities[i,r])
    
    #@constraint(m,check1_init_excess_SC[i=1:data.nbMachines,r=1:data.nbResources],excess_SC[i,r]<= 0 + 10^10 * is_excess_SC[i,r])
    
    #@constraint(m,check2_init_excess_SC[i=1:data.nbMachines,r=1:data.nbResources],excess_SC[i,r] <= sum(data.processReq[p,r] * x[i,p] for p in 1:data.nbProcess) - data.softCapacities[i,r] + 10^10 * (1 - is_excess_SC[i,r]) )    
    
    #@objective(m,Min, sum(data.weightLoadCost[r] * excess_SC[i,r] for i in 1:data.nbMachines, r in 1:data.nbResources))
    
    ###############################################
    
    
    
    ###############################################
    #               Objectif 2
    ###############################################
  
    
    @variable(m, excess_balance[i=1:data.nbMachines,j=1:data.nbBalanceCostData] >= 0, Int)
    
    @variable(m,is_excess_balance[i=1:data.nbMachines,j=1:data.nbBalanceCostData],Bin)
    
    @constraint(m,init_excess_balance[i=1:data.nbMachines,j=1:data.nbBalanceCostData],excess_balance[i,j] >= data.balanceCostDataList[j].target * ( data.hardCapacities[i,data.balanceCostDataList[j].resource1] - sum(data.processReq[p,data.balanceCostDataList[j].resource1] * x[i,p] for p in 1:data.nbProcess) ) - ( data.hardCapacities[i,data.balanceCostDataList[j].resource2] - sum(data.processReq[p ,data.balanceCostDataList[j].resource2] * x[i,p] for p in 1:data.nbProcess) ) )
    
    #@constraint(m,check1_int_1[i=1:data.nbMachines,j=1:data.nbBalanceCostData],excess_balance[i,j] <= 0 + 10^10 * is_excess_balance[i,j] )
    
    #@constraint(m,check2_int_1[i=1:data.nbMachines,j=1:data.nbBalanceCostData],excess_balance[i,j] <= data.balanceCostDataList[j].target * ( data.hardCapacities[i,data.balanceCostDataList[j].resource1] - sum(data.processReq[p,data.balanceCostDataList[j].resource1] * x[i,p] for p in 1:data.nbProcess) ) - ( data.hardCapacities[i,data.balanceCostDataList[j].resource2] - sum(data.processReq[p ,data.balanceCostDataList[j].resource2] * x[i,p] for p in 1:data.nbProcess) ) + 10^10 * (1 - is_excess_balance[i,j]))
    
    #@objective(m,Min, sum(data.balanceCostDataList[j].weight * excess_balance[m,j] for m in 1:data.nbMachines, j in 1:data.nbBalanceCostData))

    
    
    
    ###############################################
    #               Objectif 3
    ###############################################
    
    @variable(m,real[p in 1:data.nbProcess],Int)

    @constraint(m,check_move[p in 1:data.nbProcess],real[p] == data.processMoveCost[p]* (1 - x[m0[p],p]))

    #@objective(m,Min,sum(real[p] for p in 1:data.nbProcess))
    
    ###############################################
    
    
    ###############################################
    #               Objectif 4
    ###############################################
   
    
    @variable(m,nbMovesInService[i=1:data.nbServices] >= 0,Int) 
    
    @constraint(m,init_nbMovesInService[i=1:data.nbServices],nbMovesInService[i] == sum((1 - x[data.initialAssignment[p],p]) for p in 1:data.nbProcess if data.servicesProcess[p] == i)) #initialise ce tableau 
    
    @variable(m,max_nbMovesInService,Int) 
    
    @constraint(m,init_max_nbMovesInService[i=1:data.nbServices],max_nbMovesInService >= nbMovesInService[i]) 
    
    #@objective(m,Min,max_nbMovesInService) # minimiser 
    
    ###############################################

    
    ###############################################
    #               Objectif 5
    ###############################################
    
     #@objective(m,Min,sum(data.machineMoveCosts[data.initialAssignment[p],m]*x[m,p]  for p in 1:data.nbProcess,m in 1:data.nbMachines))
    
    ###############################################
    
    
    
    ###############################################
    #               Objectif Total
    ###############################################
    
    @objective(m,Min,max_nbMovesInService*data.serviceMoveWeight+sum(data.machineMoveCosts[data.initialAssignment[p],m]*x[m,p] for p in 1:data.nbProcess,m in 1:data.nbMachines)*data.machineMoveWeight + data.processMoveWeight*sum(real[p] for p in 1:data.nbProcess)+sum(data.balanceCostDataList[j].weight * excess_balance[m,j] for m in 1:data.nbMachines, j in 1:data.nbBalanceCostData)+sum(data.weightLoadCost[r] * excess_SC[i,r] for i in 1:data.nbMachines, r in 1:data.nbResources))     
    
    ###############################################
    
    
    
    
    ###############################################
    #               Contrainte 1
    ###############################################
    
    for j in 1:data.nbProcess
        @constraint(m, sum(x[i,j] for i=1:data.nbMachines) == 1 )
    end
    
    ###############################################
   
    
    
    
    ###############################################
    #               Contrainte 2
    ###############################################
    
    for i in 1:data.nbMachines
        for r in 1:data.nbResources
            @constraint(m, sum(x[i,j] * data.processReq[j,r] for j=1:data.nbProcess) <= data.hardCapacities[i,r])
        end
    end
    
    ###############################################
    
    
    
    ###############################################
    #               Contrainte 3
    ###############################################
    
    for p1 in 1:data.nbProcess
        for p2 in 1:p1-1
            if data.servicesProcess[p1]==data.servicesProcess[p2] 
                for i in 1:data.nbMachines
                        @constraint(m, x[i,p1]+x[i,p2] <= 1 )
                end
            end
        end
    end
    
    
    ###############################################

    ###############################################
    #               Contrainte 4
    ###############################################
    
    
    @variable(m, serviceLocation[1:data.nbServices,1:data.nbLocations])
    @constraint(m, conServicesLocation[s=1:data.nbServices,i=1:data.nbLocations],serviceLocation[s,i]-sum(x[m,p]
            for m=1:data.nbMachines,
                p=1:data.nbProcess
                if
                    data.servicesProcess[p] == s && data.locations[m] == i) == 0)
    @constraint(m, C4[s=1:data.nbServices], sum(1 for i=1:data.nbLocations if serviceLocation[s,i] != 0)
        >= data.spreadMin[s])
    
    ###############################################
    
    
    ###############################################
    #               Contrainte 5
    ###############################################
    #@variable(m,belong_service[p=1:data.nbProcess,s=1:data.nbServices],Bin)
    #@variable(m,is_dependence[s=1:data.nbServices,d=1:data.nbServices],Bin)
    #for i in 1:data.nbMachines
     #   for p in 1:data.nbProcess
      #      for d in 1:data.nbServices
       #         if is_dependence[data.servicesProcess[p],d] == 1
        #            @constraint(m,sum(belong_service[p_v,d]*x[data.neighborhoods[i],p_v] for p_v in 1:data.nbProcess) >= 1)
         #       end
          #  end
       # end
    #end
    
    ###############################################
    
    
    ###############################################
    #               Contrainte 6
    ###############################################
    
    #@variable(m,is_init[i=1:data.nbMachines,j=1:data.nbProcess],Bin) #Indique si la machine est l'initiale du processus considéré
    
    #for i in 1:data.nbMachines 
    #    for p in 1:data.nbProcess
    #        if m0[p] == i
    #            @constraint(m,is_init[i,p] == 1)
    #        else
    #            @constraint(m,is_init[i,p] == 0)
    #        end
    #    end
    #end
    
    #for i in 1:data.nbMachines
    #    for r in 1:data.nbResources
    #        if data.transientStatus[r]==1
    #            @constraint(m, sum(data.processReq[j,r]*is_init[i,j] for j=1:data.nbProcess) + sum(x[i,j] * data.processReq[j,r] for j=1:data.nbProcess) - sum(x[m0[j],j] * data.processReq[j,r] for j=1:data.nbProcess) <= data.hardCapacities[i,r])
    #        end
    #    end
    #end
    ###############################################
    
    
    ###############################################
    #               Résolution du modèle
    ###############################################
    
    set_optimizer_attribute(m, "seconds", 600)
   
    #print(m)        # affichage du modèle
    JuMP.optimize!(m)    # resolution à l'aide du solveur
    #########
    objVal = JuMP.objective_value(m) # to be replaced by the objective value 
    soluce = Array{Int64}(undef, data.nbProcess) # create an empty table
    for i in 1:data.nbProcess
        soluce[i] = round((sum(k*JuMP.value(x[k,i]) for k=1:data.nbMachines)))  # to be replaced by the assignment found by your model
    end
    return SolutionGoogle(soluce,objVal)
    
end




solveGoogle (generic function with 1 method)

### Cellule de test
La cellule ci-dessous permet d'exécuter le code. Elle lit la donnée, lance votre méthode solveGoogle, récupère la solution, et vérifie le respect des contraintes et son coût. 

Vous pouvez modifier le nom pour tester sur un autre jeu de donnée. 

In [4]:
# enter your instance name here 
donnee = "a1_5"  # possibles values a1_1, a1_2, a1_3, a1_4, a1_5, a2_1, a2_2, a2_3, a2_4, a2_5 

# set to yes if you want to print intermediate logs
verbose = false

main("model_"*donnee*".txt","assignment_"*donnee*".txt",verbose)


Received instance files model_a1_5.txt and assignment_a1_5.txt
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -seconds 600 -solve -quit (default strategy 1)
seconds was changed from 1e+100 to 600
Continuous objective value is 7.27578e+08 - 1.04 seconds
Cgl0003I 0 fixed, 61 tightened bounds, 324 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 72 strengthened rows, 0 substitutions
Cgl0004I processed model has 2209 rows, 12061 columns (12061 integer (12000 of which binary)) and 122389 elements
Cbc0031I 7 added rows had average density of 908
Cbc0013I At root node, 7 cuts changed objective from 7.275783e+08 to 7.275783e+08 in 3 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.118 seconds - new frequency is -100
Cbc0014I Cut generator 1 (Gomory) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.268 seconds - new frequency is -100
Cbc0

Cbc0010I After 15600 nodes, 797 on tree, 7.6109291e+08 best solution, best possibChecking the solution 
Solution received is [12, 3, 9, 8, 9, 4, 4, 4, 1, 1, 2, 6, 6, 5, 12, 5, 7, 6, 2, 5, 4, 9, 8, 9, 3, 8, 6, 3, 7, 7, 5, 1, 12, 12, 3, 6, 7, 8, 1, 8, 8, 1, 2, 1, 6, 9, 11, 12, 11, 10, 1, 3, 4, 10, 1, 7, 1, 2, 5, 2, 9, 12, 12, 10, 9, 4, 3, 8, 9, 1, 12, 3, 8, 8, 11, 2, 3, 7, 3, 11, 1, 6, 8, 6, 1, 9, 12, 11, 6, 8, 12, 7, 8, 3, 4, 4, 1, 7, 3, 1, 2, 2, 1, 1, 9, 4, 10, 4, 1, 7, 3, 6, 12, 5, 7, 4, 8, 12, 1, 4, 12, 9, 9, 4, 9, 5, 2, 1, 7, 4, 2, 3, 5, 4, 10, 4, 10, 12, 8, 10, 5, 10, 5, 4, 4, 7, 12, 4, 7, 3, 12, 12, 8, 7, 4, 5, 5, 8, 1, 8, 3, 4, 11, 10, 11, 4, 12, 7, 2, 10, 10, 4, 7, 10, 8, 12, 11, 1, 12, 9, 12, 5, 3, 4, 4, 12, 9, 11, 3, 4, 8, 7, 10, 8, 3, 9, 2, 8, 3, 1, 8, 7, 3, 11, 4, 10, 6, 7, 2, 1, 8, 1, 9, 4, 4, 3, 1, 3, 7, 11, 4, 7, 12, 10, 7, 2, 7, 12, 7, 2, 9, 1, 10, 2, 6, 4, 4, 10, 3, 10, 9, 4, 3, 8, 1, 2, 6, 5, 7, 4, 6, 9, 9, 4, 2, 1, 12, 12, 5, 5, 2, 4, 1, 7, 5, 4, 6, 6, 2, 2, 10, 1, 6,